In [1]:
import os
gpu_device = 1    # número identificador del device puede ser: 0, 1, 2, o 3
os.environ["CUDA_VISIBLE_DEVICES"]=str(gpu_device)
from numba import cuda
cuda.select_device(0)

<weakproxy at 0x7fbd84218810; to 'numba.cuda.cudadrv.driver.Device' at 0x7fbd843c9d30>

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch
import pandas as pd
import random
import math
from sklearn.metrics import classification_report, accuracy_score
from collections import defaultdict

/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_name="microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract"

In [4]:
training_args = TrainingArguments(
    output_dir='./results',          # Carpeta para guardar el modelo
    num_train_epochs=5,             # Número de épocas
    per_device_train_batch_size=16,  # Tamaño del batch
    per_device_eval_batch_size=16,   # Tamaño del batch de validación
    learning_rate=0.01, 
    weight_decay=0.05,              # Decaimiento del peso
    logging_dir='./logs',           # Carpeta para los logs
    logging_steps=10,
    evaluation_strategy="epoch",    # Evaluar durante el entrenamiento
    save_strategy="epoch"           # Guardar el modelo al final de cada época
)

training_args_all_1= TrainingArguments(
    output_dir='./results',          # Carpeta para guardar el modelo
    num_train_epochs=3,             # Número de épocas
    per_device_train_batch_size=16,  # Tamaño del batch
    per_device_eval_batch_size=16,   # Tamaño del batch de validación
    learning_rate=0.00005, 
    weight_decay=0.05,              # Decaimiento del peso
    logging_dir='./logs',           # Carpeta para los logs
    logging_steps=10,
    evaluation_strategy="epoch",    # Evaluar durante el entrenamiento
    save_strategy="epoch"           # Guardar el modelo al final de cada época
)

training_args_all_2= TrainingArguments(
    output_dir='./results',          # Carpeta para guardar el modelo
    num_train_epochs=5,             # Número de épocas
    per_device_train_batch_size=16,  # Tamaño del batch
    per_device_eval_batch_size=16,   # Tamaño del batch de validación
    learning_rate=0.00003, 
    weight_decay=0.05,              # Decaimiento del peso
    logging_dir='./logs',           # Carpeta para los logs
    logging_steps=10,
    evaluation_strategy="epoch",    # Evaluar durante el entrenamiento
    save_strategy="epoch"           # Guardar el modelo al final de cada época
)

/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [5]:
%run ../utils/data.ipynb
%run ../utils/BERT-prior.ipynb

['biopsy report', 'nodal staging ultrasound report', 'normal control or revision report', 'mammography and ultrasound', 'only mammography study', 'only ultrasound study', 'first degree', 'no family history', 'second degree', 'no prosthesis', 'yes prosthesis', 'BI-RADS 0', 'BI-RADS 1', 'BI-RADS 2', 'BI-RADS 3', 'BI-RADS 4A', 'BI-RADS 4B', 'BI-RADS 4C', 'BI-RADS 5', 'BI-RADS 6', 'ACR A', 'ACR B', 'ACR C', 'ACR D', 'unknown density mammo', 'no calcifications', 'yes calcifications', 'no ganglio', 'yes ganglio', 'fibroglandular and fat', 'heterogeneous fibroglandular', 'homogeneous fatty', 'homogeneous fibroglandular', 'unknown density echo', 'no lymph benign', 'yes lymph benign', 'no lymph suspicious', 'yes lymph suspicious', 'no cyst', 'yes cyst', 'no ectasia', 'yes ectasia', 'no nodules', 'yes nodules', 'irregular', 'lobulated', 'oval', 'round', 'unknown shape', 'circumscribed', 'indistinct', 'not circumscribed', 'spiculated', 'unknown margin', 'complex cystic and solid', 'heterogeneous'

In [6]:
dic={outputs[i]:0 for i in range(len(outputs))}
# print(targets)
# print(outputs)
for key,res in targets.items():
    print(key)
    print(res)
    # print(outputs[res])
    dic[outputs[res]]+=1
print(dic)
print(len(dic))

label
001-323-702-20221219-120748.390_tipo             2
001-323-702-20221219-120748.390_tecnica          3
001-323-702-20221219-120748.390_family           7
001-323-702-20221219-120748.390_prosthesis       9
001-323-702-20221219-120748.390_birads          13
                                                ..
999-822-503-20230123-114749_simple_cyst         38
999-822-503-20230123-114749_lymph_suspicious    36
999-822-503-20230123-114749_lymph_benign        34
999-822-503-20230123-114749_ductal_ectasia      40
999-822-503-20230123-114749_nodules_echo        42
Name: label, Length: 3027, dtype: int64


TypeError: list indices must be integers or slices, not Series

In [7]:
len(outputs)

64

In [8]:
predicted, tested=cross_validation(dataset_final,targets,dataset_final2,targets2)

EPOCH  0
                                                                                             text
529-854-912-20231002-160122_family              Question: does the patient have any family his...
420-237-263-20221121-115155_density_echo        Question: what is the breast density found in ...
192-976-987-20230612-125013.437_ductal_ectasia  Question: does the following breast medical re...
278-662-412-20211220-123944_density_echo        Question: what is the breast density found in ...
519-750-939-20220627-113153.437_tipo            Question: is the following breast medical repo...
...                                                                                           ...
278-662-412-20211220-123944_lymph_benign        Question: does the following breast medical re...
261-477-582-20230602-103817_family              Question: does the patient have any family his...
346-239-757-20230622-171503_nodules_margin      Question: what is the margin of the first nodu...
136-563-658

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map:   0%|          | 0/303 [00:00<?, ? examples/s]

⚠️ Truncation occurred!
Original length: 540, Truncated to: 512
Sample text: Question: what is the breast density found in the ultrasound study of the following breast medical report? Extra information: breast composition in ultrasound is classified into four categories: fibro ...

⚠️ Truncation occurred!
Original length: 520, Truncated to: 512
Sample text: Question: what is the breast density found in the mammography study of the following breast medical report? Extra information: breast density in mammography is classified into four categories: ACR A ( ...

⚠️ Truncation occurred!
Original length: 520, Truncated to: 512
Sample text: Question: does the patient have any family history in the following breast medical report? Extra information: family history of breast cancer is categorized based on the degree of relatives affected:  ...

⚠️ Truncation occurred!
Original length: 533, Truncated to: 512
Sample text: Question: does the following breast medical report mention any benign or n

Map: 100%|██████████| 303/303 [00:00<00:00, 432.30 examples/s]


Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2724
})
Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 303
})
Label value check:
Train labels unique: [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21), np.int64(22), np.int64(23), np.int64(24), np.int64(25), np.int64(26), np.int64(27), np.int64(28), np.int64(29), np.int64(30), np.int64(31), np.int64(32), np.int64(33), np.int64(34), np.int64(35), np.int64(36), np.int64(37), np.int64(38), np.int64(39), np.int64(40), np.int64(41), np.int64(42), np.int64(43), np.int64(44), np.int64(45), np.int64(46), np.int64(47), np.int64(48), np.int64(49), np.int64(50), np.int64(51), np.int64(52), np.int64(53), 

...

Epoch,Training Loss,Validation Loss
1,0.053700,0.378895
2,0.096700,0.246135
3,0.058200,0.249831
4,0.068300,0.244738
5,0.010100,0.224763


[40 43 56 60 53 48 63  5  0 13 26 32 21 40  6 27 34 36 42  9 38  3  2 13
 26 29 21 41  7 27 34 36 42  9 39  3  2 13 26 32 21 40  8 27 34 36 42  9
 38  3  2 15 25 33 22 40  7 27 34 36 43  9 39  3  2 18 25 33 21 40  7 27
 34 36 43 58 59 53 48  9 38  3  2 13 25 30 23 40  7 27 34 36 42  9 39  3
  2 15 25 32 24 40  7 27 34 36 42  9 38  5  2 13 26 32 21 40  7 28 34 36
 42  9 38  3  2 14 26 30 23 40  7 27 34 36 42  9 39  3  2 13 25 30 22 40
  7 27 34 36 43 56 60 49 46 63  9 38  3  2 13 25 31 20 41  7 27 34 36 43
 56 60 49 46 63  9 38  3  2 15 26 32 22 40  7 27 35 36 43 58 59 53 48  9
 38  3  2 13 25 30 22 40  7 27 34 36 43 56 60 49 45 63  9 39  3  2 14 25
 30 23 40  7 27 34 36 43 56 60 49 46 63  9 39  3  2 15 25 33 24 40  7 27
 34 36 43 58 59 49 48  9 38  5  2 14 26 30 22 40  6 27 34 36 43 56 60 49
 46 63  9 38  3  2  5  1 12 25 32 21 40  7 27 35 36 42  9 38  3  2 13 26
 32 21 40  6 27 34 36 42  9 38  3  2]
Accuracy: 0.97
Classification Report:
              precision    recall  f1-score   su

/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/ho

              precision    recall  f1-score   support

  no ganglio       0.99      0.99      0.99       179
 yes ganglio       0.86      0.86      0.86         7

    accuracy                           0.99       186
   macro avg       0.93      0.93      0.93       186
weighted avg       0.99      0.99      0.99       186


📘 Question: lymph_benign
                  precision    recall  f1-score   support

 no lymph benign       0.96      0.97      0.97       153
yes lymph benign       0.87      0.82      0.84        33

        accuracy                           0.95       186
       macro avg       0.92      0.90      0.91       186
    weighted avg       0.95      0.95      0.95       186


📘 Question: lymph_suspicious
                      precision    recall  f1-score   support

 no lymph suspicious       0.99      0.99      0.99       171
yes lymph suspicious       0.93      0.87      0.90        15

            accuracy                           0.98       186
           macro

/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu

In [8]:
evaluate_per_question(predicted, tested, DICTIONARY)


🔍 Per-question classification reports:


📘 Question: birads
              precision    recall  f1-score   support

   BI-RADS 0       0.00      0.00      0.00         3
   BI-RADS 1       0.89      1.00      0.94        24
   BI-RADS 2       0.98      0.99      0.99       107
   BI-RADS 3       0.94      0.85      0.89        20
  BI-RADS 4A       0.75      0.95      0.84        22
  BI-RADS 4B       0.50      0.25      0.33         4
  BI-RADS 4C       0.00      0.00      0.00         3
   BI-RADS 5       0.00      0.00      0.00         2
   BI-RADS 6       0.00      0.00      0.00         1

   micro avg       0.91      0.91      0.91       186
   macro avg       0.45      0.45      0.44       186
weighted avg       0.88      0.91      0.89       186


📘 Question: calcifications_benign


/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu

                    precision    recall  f1-score   support

 no calcifications       0.97      0.99      0.98        99
yes calcifications       0.99      0.97      0.98        87

          accuracy                           0.98       186
         macro avg       0.98      0.98      0.98       186
      weighted avg       0.98      0.98      0.98       186


📘 Question: density_echo
                              precision    recall  f1-score   support

      fibroglandular and fat       1.00      0.83      0.91        12
heterogeneous fibroglandular       0.92      0.97      0.95        80
           homogeneous fatty       0.00      0.00      0.00         4
  homogeneous fibroglandular       0.91      0.93      0.92        57
        unknown density echo       0.79      0.79      0.79        33

                    accuracy                           0.90       186
                   macro avg       0.72      0.71      0.71       186
                weighted avg       0.88      0.90

/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [11]:
evaluate_per_question(predicted, tested, DICTIONARY)


🔍 Per-question classification reports:


📘 Question: birads
              precision    recall  f1-score   support

   BI-RADS 0       0.67      0.67      0.67         3
   BI-RADS 1       1.00      1.00      1.00        24
   BI-RADS 2       0.99      1.00      1.00       107
   BI-RADS 3       0.95      0.95      0.95        20
  BI-RADS 4A       0.80      0.91      0.85        22
  BI-RADS 4B       0.50      0.50      0.50         4
  BI-RADS 4C       0.00      0.00      0.00         3
   BI-RADS 5       1.00      0.50      0.67         2
   BI-RADS 6       0.00      0.00      0.00         1

    accuracy                           0.94       186
   macro avg       0.66      0.61      0.63       186
weighted avg       0.93      0.94      0.93       186


📘 Question: calcifications_benign
                    precision    recall  f1-score   support

 no calcifications       0.97      0.99      0.98        99
yes calcifications       0.99      0.97      0.98        87

          accurac

/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu

In [9]:
np.save("predicted_biomed__no_prior.npy", predicted)
np.save("tested_biomed__no_prior.npy", tested)

In [ ]:
✅ Accuracy for: tipo
  biopsy report: 0.8000
  nodal staging ultrasound report: 0.0000
  normal control or revision report: 1.0000
🎯 Overall accuracy: 0.9528

✅ Accuracy for: tecnica
  mammography and ultrasound: 0.9867
  only mammography study: 0.0000
  only ultrasound study: 1.0000
🎯 Overall accuracy: 0.9811

✅ Accuracy for: family
  first degree: 0.8889
  no family history: 0.9937
  second degree: 0.7778
🎯 Overall accuracy: 0.9731

✅ Accuracy for: prosthesis
  no prosthesis: 1.0000
  yes prosthesis: 0.9000
🎯 Overall accuracy: 0.9946

✅ Accuracy for: birads
  BI-RADS 0: 0.0000
  BI-RADS 1: 1.0000
  BI-RADS 2: 1.0000
  BI-RADS 3: 1.0000
  BI-RADS 4A: 1.0000
  BI-RADS 4B: 0.5000
  BI-RADS 4C: 0.0000
  BI-RADS 5: 0.0000
  BI-RADS 6: 0.0000
🎯 Overall accuracy: 0.9409

✅ Accuracy for: density_mammo
  ACR A: 1.0000
  ACR B: 0.9512
  ACR C: 0.9872
  ACR D: 0.9286
  unknown density mammo: 0.9722
🎯 Overall accuracy: 0.9677

✅ Accuracy for: calcifications_benign
  no calcifications: 1.0000
  yes calcifications: 0.9770
🎯 Overall accuracy: 0.9892

✅ Accuracy for: ganglio_mamo
  no ganglio: 0.9944
  yes ganglio: 0.4286
🎯 Overall accuracy: 0.9731

✅ Accuracy for: density_echo
  fibroglandular and fat: 0.8333
  heterogeneous fibroglandular: 0.9750
  homogeneous fatty: 0.2500
  homogeneous fibroglandular: 0.9649
  unknown density echo: 0.9091
🎯 Overall accuracy: 0.9355

✅ Accuracy for: lymph_benign
  no lymph benign: 0.9673
  yes lymph benign: 0.8485
🎯 Overall accuracy: 0.9462

✅ Accuracy for: lymph_suspicious
  no lymph suspicious: 0.9942
  yes lymph suspicious: 0.6667
🎯 Overall accuracy: 0.9677

✅ Accuracy for: simple_cyst
  no cyst: 0.9703
  yes cyst: 0.9882
🎯 Overall accuracy: 0.9785

✅ Accuracy for: ductal_ectasia
  no ectasia: 1.0000
  yes ectasia: 0.9375
🎯 Overall accuracy: 0.9946

✅ Accuracy for: nodules_echo
  no nodules: 0.9231
  yes nodules: 0.9512
🎯 Overall accuracy: 0.9355

✅ Accuracy for: nodules_shape
  irregular: 0.0000
  lobulated: 0.8462
  oval: 0.9600
  round: 0.0000
  unknown shape: 0.9250
🎯 Overall accuracy: 0.8780

✅ Accuracy for: nodules_margin
  circumscribed: 0.9394
  indistinct: 0.3333
  not circumscribed: 0.0000
  spiculated: 0.0000
  unknown margin: 0.9524
🎯 Overall accuracy: 0.8780

✅ Accuracy for: nodules_echogenicity
  complex cystic and solid: 0.0000
  heterogeneous: 0.0000
  hypoechoic: 1.0000
  isoechoic: 0.0000
  unknown echogenicity: 0.9310
🎯 Overall accuracy: 0.9146

✅ Accuracy for: nodules_known
  no known: 0.8421
  yes known: 0.8182
🎯 Overall accuracy: 0.8293

✅ Accuracy for: nodules_stable
  grown stable: 0.0000
  shrunk stable: 0.0000
  unknown stable: 0.0000
  yes stable: 0.9429
🎯 Overall accuracy: 0.7500

In [ ]:
✅ Accuracy for: tipo
  biopsy report: 0.8500
  nodal staging ultrasound report: 0.3333
  normal control or revision report: 1.0000
🎯 Overall accuracy: 0.9670

✅ Accuracy for: tecnica
  mammography and ultrasound: 0.9933
  only mammography study: 0.0000
  only ultrasound study: 0.9833
🎯 Overall accuracy: 0.9811

✅ Accuracy for: family
  first degree: 0.9444
  no family history: 1.0000
  second degree: 0.8889
🎯 Overall accuracy: 0.9892

✅ Accuracy for: prosthesis
  no prosthesis: 1.0000
  yes prosthesis: 0.9000
🎯 Overall accuracy: 0.9946

✅ Accuracy for: birads
  BI-RADS 0: 0.6667
  BI-RADS 1: 1.0000
  BI-RADS 2: 1.0000
  BI-RADS 3: 1.0000
  BI-RADS 4A: 0.9545
  BI-RADS 4B: 1.0000
  BI-RADS 4C: 0.3333
  BI-RADS 5: 0.0000
  BI-RADS 6: 0.0000
  unknown BI-RADS: [No samples]
🎯 Overall accuracy: 0.9624

✅ Accuracy for: density_mammo
  ACR A: 0.6667
  ACR B: 0.9512
  ACR C: 0.9872
  ACR D: 0.9643
  unknown density mammo: 0.9722
🎯 Overall accuracy: 0.9677

✅ Accuracy for: calcifications_benign
  no calcifications: 0.9899
  yes calcifications: 0.9655
🎯 Overall accuracy: 0.9785

✅ Accuracy for: ganglio_mamo
  no ganglio: 0.9944
  yes ganglio: 0.8571
🎯 Overall accuracy: 0.9892

✅ Accuracy for: density_echo
  fibroglandular and fat: 0.9167
  heterogeneous fibroglandular: 0.9750
  homogeneous fatty: 0.5000
  homogeneous fibroglandular: 0.9825
  unknown density echo: 0.8788
🎯 Overall accuracy: 0.9462

✅ Accuracy for: lymph_benign
  no lymph benign: 0.9804
  yes lymph benign: 0.7879
🎯 Overall accuracy: 0.9462

✅ Accuracy for: lymph_suspicious
  no lymph suspicious: 0.9942
  yes lymph suspicious: 0.6667
🎯 Overall accuracy: 0.9677

✅ Accuracy for: simple_cyst
  no cyst: 0.9802
  yes cyst: 0.9882
🎯 Overall accuracy: 0.9839

✅ Accuracy for: ductal_ectasia
  no ectasia: 1.0000
  yes ectasia: 0.9375
🎯 Overall accuracy: 0.9946

In [12]:
evaluate_per_question(predicted, tested, DICTIONARY)


🔍 Per-question classification reports:


📘 Question: birads
                 precision    recall  f1-score   support

      BI-RADS 0       0.00      0.00      0.00         3
      BI-RADS 1       0.96      1.00      0.98        24
      BI-RADS 2       0.90      0.94      0.92       107
      BI-RADS 3       0.65      0.55      0.59        20
     BI-RADS 4A       0.45      0.64      0.53        22
     BI-RADS 4B       0.00      0.00      0.00         4
     BI-RADS 4C       0.00      0.00      0.00         3
      BI-RADS 5       0.00      0.00      0.00         2
      BI-RADS 6       0.00      0.00      0.00         1
unknown BI-RADS       0.00      0.00      0.00         0

      micro avg       0.81      0.81      0.81       186
      macro avg       0.30      0.31      0.30       186
   weighted avg       0.77      0.81      0.78       186


📘 Question: calcifications_benign
                    precision    recall  f1-score   support

 no calcifications       0.94      0.99   

/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resul

              precision    recall  f1-score   support

  no ectasia       0.99      1.00      0.99       170
 yes ectasia       1.00      0.88      0.93        16

    accuracy                           0.99       186
   macro avg       0.99      0.94      0.96       186
weighted avg       0.99      0.99      0.99       186


📘 Question: family
                   precision    recall  f1-score   support

     first degree       0.94      0.94      0.94        18
no family history       1.00      1.00      1.00       159
    second degree       0.89      0.89      0.89         9

         accuracy                           0.99       186
        macro avg       0.94      0.94      0.94       186
     weighted avg       0.99      0.99      0.99       186


📘 Question: ganglio_mamo
              precision    recall  f1-score   support

  no ganglio       0.99      0.99      0.99       179
 yes ganglio       0.83      0.71      0.77         7

    accuracy                           0.98    

/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu